In [1]:
pip install openai==1.11.1 speechrecognition pyaudio gtts opencv-python


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install playsound==1.2.2
!pip install pyttsx3



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install openai-whisper


Defaulting to user installation because normal site-packages is not writeable


In [24]:
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   --------- ------------------------------ 2.4/10.5 MB 12.2 MB/s eta 0:00:01
   ------------------- -------------------- 5.0/10.5 MB 12.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.5 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 5.6 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 882.6 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 931.2 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 931.2 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 868.0 kB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 868.0 kB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 849.7 kB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 830.6 kB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 794.3 kB/s eta 0:00:00


In [ ]:
import csv
import pyttsx3
import speech_recognition as sr
import cv2
import os
from openai import OpenAI

#  Set your OpenAI API Key
openai_api_key = ""  # Replace with your actual OpenAI API Key
client = OpenAI(api_key=openai_api_key)

#  Text-to-speech function
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

#  Load questions and answers for a specific role
def load_questions_from_csv(filename, role):
    questions = []
    with open(filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["role"].strip().lower() == role.strip().lower():
                questions.append((row["question"], row["answer"]))
    return questions

#  Record audio and transcribe using Google API
def record_audio_response():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print(" Listening for your answer...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(" Your answer:", text)
            return text
        except sr.UnknownValueError:
            print(" Could not understand audio.")
            return ""
        except sr.RequestError:
            print(" Could not connect to Google API.")
            return ""

#  Record webcam video with OpenCV (10 sec or until 'q')
import cv2
import time
import speech_recognition as sr

def record_audio_response():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for your answer...")
        audio = recognizer.listen(source)
        try:
            response = recognizer.recognize_google(audio)
            print("Your answer:", response)
            return response
        except sr.UnknownValueError:
            print("Could not understand audio.")
            return ""
        except sr.RequestError:
            print("Google Speech API error.")
            return ""

#  Capture video with live preview

def capture_video_with_preview(filename="response.avi", duration=10):
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Camera failed to open.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(filename, fourcc, 20.0, (640, 480))

    cv2.namedWindow("Live Recording", cv2.WINDOW_NORMAL)
    cv2.resizeWindow(" Live Recording", 640, 480)

    start_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        cv2.imshow("🎥 Live Recording", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if time.time() - start_time > duration:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(" Video recording done.")





# Get AI feedback (2-line review + score + correct answer)
def get_feedback(question, answer, correct_answer):
    prompt = (
        f"You're an interview coach.\n"
        f"Question: {question}\n"
        f"Candidate's Answer: {answer}\n"
        f"Correct Answer: {correct_answer}\n"
        f"Give a 2-line assessment of the answer, rate it out of 10, and mention the correct answer again."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(" Feedback error:", e)
        return "Feedback not available."

# Main Interview Flow
def main():
    print("AI Mock Interview Coach")
    role = input("Enter the job role you want to be interviewed for: ").strip()

    filename = "interview_questions_dataset.csv"
    questions = load_questions_from_csv(filename, role)

    if not questions:
        print("No questions found for this role.")
        return

    for i, (question, correct_answer) in enumerate(questions[:3], 1):
        print(f"\n Question {i}: {question}")
        speak(question)

        input_mode = input("Choose input mode: 1) Speak 2) Type: ").strip()

        answer = ""
        if input_mode == "1":
            answer = capture_video_then_record_audio(video_filename=f"response_q{i}.avi", duration=10)

        elif input_mode == "2":
            answer = input("Type your answer: ").strip()
        else:
            print("Invalid input. Skipping question.")
            continue

        if not answer:
            print(" No answer provided. Skipping.")
            continue

        feedback = get_feedback(question, answer, correct_answer)
        print(f"\nFeedback:\n{feedback}")
        speak(feedback)

# ✅ Run
if __name__ == "__main__":
    main()


🎯 AI Mock Interview Coach


Enter the job role you want to be interviewed for:  Software Engineer



🧠 Question 1: Explain the difference between a process and a thread.
